In [1]:
import json
import torch

In [3]:
from MyDataset import MyDataset
from torch.utils.data import DataLoader

In [ ]:
lama_embedding_path = '../llama_embedding/CL/tumor_CL_E3_finetuned_5layers/dataset_tensor/'
bert_embedding_path = '../bert_embedding/tumor_finetuned/dataset_tensor/'
roberta_embedding_path = '../roberta_embedding/tumor_finetuned/dataset_tensor/'

In [5]:
train_data = MyDataset('train', lama_embedding_path, bert_embedding_path, roberta_embedding_path)
test_data =MyDataset('test', lama_embedding_path, bert_embedding_path, roberta_embedding_path)

In [6]:
train_data = DataLoader(train_data, batch_size=1, shuffle=False)
test_data = DataLoader(test_data, batch_size=1, shuffle=False) 

In [7]:
for i, data in enumerate(train_data):
    print(data)
    break

[tensor([[[ 1.3761,  0.2127, -0.9604,  ..., -1.2733,  0.2727,  0.8831],
         [ 1.7192, -0.6819, -0.1220,  ..., -1.2109,  0.6658, -0.7323],
         [-0.5107,  0.4175, -0.2356,  ..., -0.0969,  0.0928,  0.3295],
         [-0.7307,  0.2628, -0.3072,  ..., -0.3696,  0.3892,  0.3428],
         [-0.4344,  0.2354, -0.2602,  ..., -0.1401,  0.1015,  0.3727]]]), tensor([[-0.8550,  0.6309, -0.6627,  ..., -0.5210,  0.2564, -0.3477]]), tensor([[ 1.7732,  0.9521, -0.7721,  ..., -0.5331, -0.3238, -0.3959]]), tensor([1])]


In [8]:
l_rep, b_rep, r_rep, label = data

In [9]:
b_rep.squeeze().shape

torch.Size([1024])

In [10]:
l_rep.squeeze().shape, b_rep.squeeze().shape, r_rep.squeeze().shape, label.item()

(torch.Size([5, 4096]), torch.Size([1024]), torch.Size([1024]), 1)

In [11]:
# reps 별 코사인 유사도 평균 구해보기
def get_l1_distance(x1, x2):
    return ((x1 - x2).abs()).sum()
def get_l2_distance(x1, x2):
    return ((x1 - x2)**2).sum()**.5
def get_cosine_similarity(x1, x2):
    return (x1 * x2).sum() / ((x1**2).sum()**.5 * (x2**2).sum()**.5)

In [12]:
#get_cosine_similarity(reps_dict[0]['lama'][0].flatten() , reps_dict[0]['lama'][1].flatten()).item()

# Label 끼리의 코사인 유사도
## Train Data

In [13]:
reps_dict = {i : {'lama' : [],
                  'bert' : [],
                'roberta' : [] } for i in range(6)}
cnt = 0
for data in train_data:
    cnt += 1
    l_rep, b_rep, r_rep, label = data
    l_rep = l_rep.squeeze()
    b_rep = b_rep.squeeze()
    r_rep = r_rep.squeeze()
    label = label.item()
    reps_dict[label]['lama'].append(l_rep)
    reps_dict[label]['bert'].append(b_rep)
    reps_dict[label]['roberta'].append(r_rep)
cnt == len(train_data)

True

In [14]:
all_dict = {i :{'lama' : [] ,'bert': [],'roberta': []} for i in range(6)}
for idx in range(6):
    # lama, bert, roberta 각각의 평균 코사인 유사도 구하기
    lama_cosine = []
    bert_cosine = []
    roberta_cosine = []
    for i in range(len(reps_dict[idx]['lama'])):
        for j in range(i+1, len(reps_dict[idx]['lama'])):
            lama_cosine.append(get_cosine_similarity(reps_dict[idx]['lama'][i].flatten() , reps_dict[idx]['lama'][j].flatten()).item())
            bert_cosine.append(get_cosine_similarity(reps_dict[idx]['bert'][i].flatten() , reps_dict[idx]['bert'][j].flatten()).item())
            roberta_cosine.append(get_cosine_similarity(reps_dict[idx]['roberta'][i].flatten() , reps_dict[idx]['roberta'][j].flatten()).item())

    all_dict[idx]['lama'] = torch.tensor(lama_cosine)
    all_dict[idx]['bert'] = torch.tensor(bert_cosine)
    all_dict[idx]['roberta'] = torch.tensor(roberta_cosine)

In [15]:
for i in range(6):
    print(f'Label {i} :')
    print(f'Lama mean : {all_dict[i]["lama"].mean()}, std : {all_dict[i]["lama"].std()}, min : {all_dict[i]["lama"].min()}, max : {all_dict[i]["lama"].max()}')
    print(f'Bert mean : {all_dict[i]["bert"].mean()}, std : {all_dict[i]["bert"].std()}, min : {all_dict[i]["bert"].min()}, max : {all_dict[i]["bert"].max()}')
    print(f'Roberta mean : {all_dict[i]["roberta"].mean()}, std : {all_dict[i]["roberta"].std()}, min : {all_dict[i]["roberta"].min()}, max : {all_dict[i]["roberta"].max()}')
    print()

Label 0 :
Lama mean : 0.9617108702659607, std : 0.14054253697395325, min : 0.015174771659076214, max : 0.9996907114982605
Bert mean : 0.9680062532424927, std : 0.06482237577438354, min : 0.5762061476707458, max : 0.9999645352363586
Roberta mean : 0.7679231762886047, std : 0.3273807466030121, min : -0.14232468605041504, max : 0.9999403357505798

Label 1 :
Lama mean : 0.9832085967063904, std : 0.010509264655411243, min : 0.9182170033454895, max : 0.9996959567070007
Bert mean : 0.967339038848877, std : 0.11414065212011337, min : 0.08446459472179413, max : 0.9999260902404785
Roberta mean : 0.9764479398727417, std : 0.1192246600985527, min : 0.14114268124103546, max : 0.9999465346336365

Label 2 :
Lama mean : 0.9851194620132446, std : 0.009300495497882366, min : 0.9220899939537048, max : 0.9993664026260376
Bert mean : 0.9849059581756592, std : 0.014416240155696869, min : 0.9077816605567932, max : 1.0
Roberta mean : 0.9430029988288879, std : 0.08131575584411621, min : 0.4610406756401062, max

## Test Data

In [16]:
reps_dict = {i : {'lama' : [],
                  'bert' : [],
                'roberta' : [] } for i in range(6)}
cnt = 0
for data in test_data:
    cnt += 1
    l_rep, b_rep, r_rep, label = data
    l_rep = l_rep.squeeze()
    b_rep = b_rep.squeeze()
    r_rep = r_rep.squeeze()
    label = label.item()
    reps_dict[label]['lama'].append(l_rep)
    reps_dict[label]['bert'].append(b_rep)
    reps_dict[label]['roberta'].append(r_rep)
cnt == len(test_data)

True

In [17]:
all_dict = {i :{'lama' : [] ,'bert': [],'roberta': []} for i in range(6)}
for idx in range(6):
    # lama, bert, roberta 각각의 평균 코사인 유사도 구하기
    lama_cosine = []
    bert_cosine = []
    roberta_cosine = []
    for i in range(len(reps_dict[idx]['lama'])):
        for j in range(i+1, len(reps_dict[idx]['lama'])):
            lama_cosine.append(get_cosine_similarity(reps_dict[idx]['lama'][i].flatten() , reps_dict[idx]['lama'][j].flatten()).item())
            bert_cosine.append(get_cosine_similarity(reps_dict[idx]['bert'][i].flatten() , reps_dict[idx]['bert'][j].flatten()).item())
            roberta_cosine.append(get_cosine_similarity(reps_dict[idx]['roberta'][i].flatten() , reps_dict[idx]['roberta'][j].flatten()).item())

    all_dict[idx]['lama'] = torch.tensor(lama_cosine)
    all_dict[idx]['bert'] = torch.tensor(bert_cosine)
    all_dict[idx]['roberta'] = torch.tensor(roberta_cosine)

In [18]:
for i in range(6):
    print(f'Label {i} :')
    print(f'Lama mean : {all_dict[i]["lama"].mean()}, std : {all_dict[i]["lama"].std()}, min : {all_dict[i]["lama"].min()}, max : {all_dict[i]["lama"].max()}')
    print(f'Bert mean : {all_dict[i]["bert"].mean()}, std : {all_dict[i]["bert"].std()}, min : {all_dict[i]["bert"].min()}, max : {all_dict[i]["bert"].max()}')
    print(f'Roberta mean : {all_dict[i]["roberta"].mean()}, std : {all_dict[i]["roberta"].std()}, min : {all_dict[i]["roberta"].min()}, max : {all_dict[i]["roberta"].max()}')
    print()

Label 0 :
Lama mean : 0.7692450881004333, std : 0.3749873638153076, min : 0.006966863293200731, max : 0.9994299411773682
Bert mean : 0.8336034417152405, std : 0.3186386525630951, min : 0.08265118300914764, max : 0.9998219013214111
Roberta mean : 0.6699555516242981, std : 0.3867779076099396, min : -0.21332025527954102, max : 0.9997442364692688

Label 1 :
Lama mean : 0.8680761456489563, std : 0.27354198694229126, min : 0.027804411947727203, max : 0.999306857585907
Bert mean : 0.7791711091995239, std : 0.4013894498348236, min : -0.16923663020133972, max : 0.9998753070831299
Roberta mean : 0.9898001551628113, std : 0.011732799932360649, min : 0.9500171542167664, max : 0.9999515414237976

Label 2 :
Lama mean : 0.9493721127510071, std : 0.05563623458147049, min : 0.7215757369995117, max : 0.9934617877006531
Bert mean : 0.9522998332977295, std : 0.059857457876205444, min : 0.6458935737609863, max : 0.9971312284469604
Roberta mean : 0.6465266346931458, std : 0.39903706312179565, min : -0.11720

## Llama Layer 별 코사인 유사도

### Train

In [19]:
# reps_dict = {i : {'lama' : {j : [] for j in range(5)},
#                   'bert' : [],
#                 'roberta' : [] } for i in range(6)}
# cnt = 0
# for data in train_data:
#     cnt += 1
#     l_rep, b_rep, r_rep, label = data
#     l_rep = l_rep.squeeze()
#     l_rep_0 = l_rep[0]
#     l_rep_1 = l_rep[1]
#     l_rep_2 = l_rep[2]
#     l_rep_3 = l_rep[3]
#     l_rep_4 = l_rep[4]
#     b_rep = b_rep.squeeze()
#     r_rep = r_rep.squeeze()
#     label = label.item()
#     reps_dict[label]['lama'][0].append(l_rep_0)
#     reps_dict[label]['lama'][1].append(l_rep_1)
#     reps_dict[label]['lama'][2].append(l_rep_2)
#     reps_dict[label]['lama'][3].append(l_rep_3)
#     reps_dict[label]['lama'][4].append(l_rep_4)
#     reps_dict[label]['bert'].append(b_rep)
#     reps_dict[label]['roberta'].append(r_rep)

# cnt == len(train_data)

In [20]:
# all_dict = {i : {'lama' : {j : [] for j in range(5)},
#                   'bert' : [],
#                 'roberta' : [] } for i in range(6)}
# for idx in range(6):
#     # bert, roberta 각각의 평균 코사인 유사도 구하기
#     bert_cosine = []
#     roberta_cosine = []
#     for i in range(len(reps_dict[idx]['bert'])):
#         for j in range(i+1, len(reps_dict[idx]['bert'])):
#             bert_cosine.append(get_cosine_similarity(reps_dict[idx]['bert'][i].flatten() , reps_dict[idx]['bert'][j].flatten()).item())
#             roberta_cosine.append(get_cosine_similarity(reps_dict[idx]['roberta'][i].flatten() , reps_dict[idx]['roberta'][j].flatten()).item())

#     all_dict[idx]['bert'] = torch.tensor(bert_cosine)
#     all_dict[idx]['roberta'] = torch.tensor(roberta_cosine)
# # llama layer별
# for idx in range(6):
#     for i in range(5): # lama layer별
#         lama_cosine = []
#         for j in range(len(reps_dict[idx]['lama'][i])):
#             for k in range(j+1, len(reps_dict[idx]['lama'][i])):
#                 lama_cosine.append(get_cosine_similarity(reps_dict[idx]['lama'][i][j].flatten(),reps_dict[idx]['lama'][i][k].flatten()).item())
        
#         all_dict[idx]['lama'][i] = torch.tensor(lama_cosine)

In [21]:
# all_dict

In [22]:
# reps_dict[idx]['lama']

In [23]:
# for i in range(6):
#     print(f'Label {i} :')
#     # lama layer 별
#     print(f'lama layer 0  mean : {all_dict[i]["lama"][0].mean()} std : {all_dict[i]["lama"][0].std()} min : {all_dict[i]["lama"][0].min()} max : {all_dict[i]["lama"][0].max()}')
#     print(f'lama layer 1  mean : {all_dict[i]["lama"][1].mean()} std : {all_dict[i]["lama"][1].std()} min : {all_dict[i]["lama"][1].min()} max : {all_dict[i]["lama"][1].max()}')
#     print(f'lama layer 2  mean : {all_dict[i]["lama"][2].mean()} std : {all_dict[i]["lama"][2].std()} min : {all_dict[i]["lama"][2].min()} max : {all_dict[i]["lama"][2].max()}')
#     print(f'lama layer 3  mean : {all_dict[i]["lama"][3].mean()} std : {all_dict[i]["lama"][3].std()} min : {all_dict[i]["lama"][3].min()} max : {all_dict[i]["lama"][3].max()}')
#     print(f'lama layer 4  mean : {all_dict[i]["lama"][4].mean()} std : {all_dict[i]["lama"][4].std()} min : {all_dict[i]["lama"][4].min()} max : {all_dict[i]["lama"][4].max()}')


### Test

In [24]:
# reps_dict = {i : {'lama' : {j : [] for j in range(5)},
#                   'bert' : [],
#                 'roberta' : [] } for i in range(6)}
# cnt = 0
# for data in test_data:
#     cnt += 1
#     l_rep, b_rep, r_rep, label = data
#     l_rep = l_rep.squeeze()
#     l_rep_0 = l_rep[0]
#     l_rep_1 = l_rep[1]
#     l_rep_2 = l_rep[2]
#     l_rep_3 = l_rep[3]
#     l_rep_4 = l_rep[4]
#     b_rep = b_rep.squeeze()
#     r_rep = r_rep.squeeze()
#     label = label.item()
#     reps_dict[label]['lama'][0].append(l_rep_0)
#     reps_dict[label]['lama'][1].append(l_rep_1)
#     reps_dict[label]['lama'][2].append(l_rep_2)
#     reps_dict[label]['lama'][3].append(l_rep_3)
#     reps_dict[label]['lama'][4].append(l_rep_4)
#     reps_dict[label]['bert'].append(b_rep)
#     reps_dict[label]['roberta'].append(r_rep)

# cnt == len(test_data)

In [25]:
# all_dict = {i : {'lama' : {j : [] for j in range(5)},
#                   'bert' : [],
#                 'roberta' : [] } for i in range(6)}
# for idx in range(6):
#     # bert, roberta 각각의 평균 코사인 유사도 구하기
#     bert_cosine = []
#     roberta_cosine = []
#     for i in range(len(reps_dict[idx]['bert'])):
#         for j in range(i+1, len(reps_dict[idx]['bert'])):
#             bert_cosine.append(get_cosine_similarity(reps_dict[idx]['bert'][i].flatten() , reps_dict[idx]['bert'][j].flatten()).item())
#             roberta_cosine.append(get_cosine_similarity(reps_dict[idx]['roberta'][i].flatten() , reps_dict[idx]['roberta'][j].flatten()).item())

#     all_dict[idx]['bert'] = torch.tensor(bert_cosine)
#     all_dict[idx]['roberta'] = torch.tensor(roberta_cosine)

In [26]:
# for idx in range(6):
#     for i in range(5): # lama layer별
#         lama_cosine = []
#         for j in range(len(reps_dict[idx]['lama'][i])):
#             for k in range(j+1, len(reps_dict[idx]['lama'][i])):
#                 lama_cosine.append(get_cosine_similarity(reps_dict[idx]['lama'][i][j].flatten(),reps_dict[idx]['lama'][i][k].flatten()).item())
        
#         all_dict[idx]['lama'][i] = torch.tensor(lama_cosine)

In [27]:
# for i in range(6):
#     print(f'Label {i} :')
#     # lama layer 별
#     print(f'lama layer 0  mean : {all_dict[i]["lama"][0].mean()} std : {all_dict[i]["lama"][0].std()} min : {all_dict[i]["lama"][0].min()} max : {all_dict[i]["lama"][0].max()}')
#     print(f'lama layer 1  mean : {all_dict[i]["lama"][1].mean()} std : {all_dict[i]["lama"][1].std()} min : {all_dict[i]["lama"][1].min()} max : {all_dict[i]["lama"][1].max()}')
#     print(f'lama layer 2  mean : {all_dict[i]["lama"][2].mean()} std : {all_dict[i]["lama"][2].std()} min : {all_dict[i]["lama"][2].min()} max : {all_dict[i]["lama"][2].max()}')
#     print(f'lama layer 3  mean : {all_dict[i]["lama"][3].mean()} std : {all_dict[i]["lama"][3].std()} min : {all_dict[i]["lama"][3].min()} max : {all_dict[i]["lama"][3].max()}')
#     print(f'lama layer 4  mean : {all_dict[i]["lama"][4].mean()} std : {all_dict[i]["lama"][4].std()} min : {all_dict[i]["lama"][4].min()} max : {all_dict[i]["lama"][4].max()}')


# 다른 라벨들과의 코사인 유사도 비교
## TrainData

In [28]:
reps_dict = {i : {'lama' : [],
                  'bert' : [],
                'roberta' : [] } for i in range(6)}
cnt = 0
for data in train_data:
    cnt += 1
    l_rep, b_rep, r_rep, label = data
    l_rep = l_rep.squeeze()
    b_rep = b_rep.squeeze()
    r_rep = r_rep.squeeze()
    label = label.item()
    reps_dict[label]['lama'].append(l_rep)
    reps_dict[label]['bert'].append(b_rep)
    reps_dict[label]['roberta'].append(r_rep)
cnt == len(train_data)

True

In [29]:
all_dict = {i : {j :{'lama' : [] ,'bert': [],'roberta': []}  for j in range(6) if i != j } for i in range(6)}
for i in range(6):
    # lama, bert, roberta 각각의 평균 코사인 유사도 구하기

    for j in range(6):
        lama_cosine = []
        bert_cosine = []
        roberta_cosine = []
        if i != j :
            for l in range(len(reps_dict[i]['lama'])):
                for m in range(len(reps_dict[j]['lama'])):
                    lama_cosine.append(get_cosine_similarity(reps_dict[i]['lama'][l].flatten() , reps_dict[j]['lama'][m].flatten()).item())
                    bert_cosine.append(get_cosine_similarity(reps_dict[i]['bert'][l].flatten() , reps_dict[j]['bert'][m].flatten()).item())
                    roberta_cosine.append(get_cosine_similarity(reps_dict[i]['roberta'][l].flatten() , reps_dict[j]['roberta'][m].flatten()).item())
            
            all_dict[i][j]['lama'] = torch.tensor(lama_cosine)
            all_dict[i][j]['bert'] = torch.tensor(bert_cosine)
            all_dict[i][j]['roberta'] = torch.tensor(roberta_cosine)
        else:
            continue

In [30]:
# i, j 레이블 간의 평균 코사인 유사도 구하기
for i in range(6):
    for j in range(6):
        if i != j:
            print(f'Label {i} - Label {j} :')
            print(f'Lama mean : {all_dict[i][j]["lama"].mean()}, std : {all_dict[i][j]["lama"].std()}, min : {all_dict[i][j]["lama"].min()}, max : {all_dict[i][j]["lama"].max()}')
            print(f'Bert mean : {all_dict[i][j]["bert"].mean()}, std : {all_dict[i][j]["bert"].std()}, min : {all_dict[i][j]["bert"].min()}, max : {all_dict[i][j]["bert"].max()}')
            print(f'Roberta mean : {all_dict[i][j]["roberta"].mean()}, std : {all_dict[i][j]["roberta"].std()}, min : {all_dict[i][j]["roberta"].min()}, max : {all_dict[i][j]["roberta"].max()}')
            print()

Label 0 - Label 1 :
Lama mean : 0.037323351949453354, std : 0.008496801368892193, min : 0.009072191081941128, max : 0.08510429412126541
Bert mean : -0.018838658928871155, std : 0.06299740821123123, min : -0.28486230969429016, max : 0.41429656744003296
Roberta mean : 0.09390362352132797, std : 0.2708650529384613, min : -0.15240997076034546, max : 0.9793902039527893

Label 0 - Label 2 :
Lama mean : 0.0032276418060064316, std : 0.008141499012708664, min : -0.04681554064154625, max : 0.04380728676915169
Bert mean : -0.3624917268753052, std : 0.04028254747390747, min : -0.4767549932003021, max : -0.07394306361675262
Roberta mean : -0.11761458963155746, std : 0.05398637428879738, min : -0.37233245372772217, max : 0.5991447567939758

Label 0 - Label 3 :
Lama mean : 0.031872574239969254, std : 0.10059945285320282, min : 0.0024723615497350693, max : 0.9944412112236023
Bert mean : 0.01212023664265871, std : 0.07806583493947983, min : -0.27485862374305725, max : 0.40978315472602844
Roberta mean :

### lama layer 별

### train

In [31]:
# reps_dict = {i : {'lama' : {j : [] for j in range(5)},
#                   'bert' : [],
#                 'roberta' : [] } for i in range(6)}
# cnt = 0
# for data in train_data:
#     cnt += 1
#     l_rep, b_rep, r_rep, label = data
#     l_rep = l_rep.squeeze()
#     l_rep_0 = l_rep[0]
#     l_rep_1 = l_rep[1]
#     l_rep_2 = l_rep[2]
#     l_rep_3 = l_rep[3]
#     l_rep_4 = l_rep[4]
#     b_rep = b_rep.squeeze()
#     r_rep = r_rep.squeeze()
#     label = label.item()
#     reps_dict[label]['lama'][0].append(l_rep_0)
#     reps_dict[label]['lama'][1].append(l_rep_1)
#     reps_dict[label]['lama'][2].append(l_rep_2)
#     reps_dict[label]['lama'][3].append(l_rep_3)
#     reps_dict[label]['lama'][4].append(l_rep_4)
#     reps_dict[label]['bert'].append(b_rep)
#     reps_dict[label]['roberta'].append(r_rep)

# cnt == len(train_data)

In [32]:
# all_dict = {i : {j :{'lama' : {k : [] for k in range(5)}}  for j in range(6) if i != j } for i in range(6)}
# for main_label in range(6):
#     for next_label in range(6):
#         for layer_num in range(5):
#             lama_cosine = []
#             if main_label != next_label:
#                 for l in range(len(reps_dict[main_label]['lama'][layer_num])):
#                     for k in range(len(reps_dict[next_label]['lama'][layer_num])):
#                         lama_cosine.append(get_cosine_similarity(reps_dict[main_label]['lama'][layer_num][l], reps_dict[next_label]['lama'][layer_num][k]))
            
#                 all_dict[main_label][next_label]['lama'][layer_num] = torch.tensor(lama_cosine)

In [33]:
# # i, j 레이블 간의 평균 코사인 유사도 구하기
# for i in range(6):
#     for j in range(6):
#         if i != j:
#             print(f'Label {i} - Label {j} _ Layer : {0}')
#             print(f'Lama mean : {all_dict[i][j]["lama"][0].mean()}, std : {all_dict[i][j]["lama"][0].std()}, min : {all_dict[i][j]["lama"][0].min()}, max : {all_dict[i][j]["lama"][0].max()}')
            
#             print(f'Label {i} - Label {j} _ Layer : {1}')
#             print(f'Lama mean : {all_dict[i][j]["lama"][1].mean()}, std : {all_dict[i][j]["lama"][1].std()}, min : {all_dict[i][j]["lama"][1].min()}, max : {all_dict[i][j]["lama"][1].max()}')

#             print(f'Label {i} - Label {j} _ Layer : {2}')
#             print(f'Lama mean : {all_dict[i][j]["lama"][2].mean()}, std : {all_dict[i][j]["lama"][2].std()}, min : {all_dict[i][j]["lama"][2].min()}, max : {all_dict[i][j]["lama"][2].max()}')

#             print(f'Label {i} - Label {j} _ Layer : {3}')
#             print(f'Lama mean : {all_dict[i][j]["lama"][3].mean()}, std : {all_dict[i][j]["lama"][3].std()}, min : {all_dict[i][j]["lama"][3].min()}, max : {all_dict[i][j]["lama"][3].max()}')

#             print(f'Label {i} - Label {j} _ Layer : {4}')
#             print(f'Lama mean : {all_dict[i][j]["lama"][4].mean()}, std : {all_dict[i][j]["lama"][4].std()}, min : {all_dict[i][j]["lama"][4].min()}, max : {all_dict[i][j]["lama"][4].max()}')


# Test

In [34]:
# reps_dict = {i : {'lama' : {j : [] for j in range(5)},
#                   'bert' : [],
#                 'roberta' : [] } for i in range(6)}
# cnt = 0
# for data in test_data:
#     cnt += 1
#     l_rep, b_rep, r_rep, label = data
#     l_rep = l_rep.squeeze()
#     l_rep_0 = l_rep[0]
#     l_rep_1 = l_rep[1]
#     l_rep_2 = l_rep[2]
#     l_rep_3 = l_rep[3]
#     l_rep_4 = l_rep[4]
#     b_rep = b_rep.squeeze()
#     r_rep = r_rep.squeeze()
#     label = label.item()
#     reps_dict[label]['lama'][0].append(l_rep_0)
#     reps_dict[label]['lama'][1].append(l_rep_1)
#     reps_dict[label]['lama'][2].append(l_rep_2)
#     reps_dict[label]['lama'][3].append(l_rep_3)
#     reps_dict[label]['lama'][4].append(l_rep_4)
#     reps_dict[label]['bert'].append(b_rep)
#     reps_dict[label]['roberta'].append(r_rep)

# cnt == len(test_data)

In [35]:
# all_dict = {i : {j :{'lama' : {k : [] for k in range(5)}}  for j in range(6) if i != j } for i in range(6)}
# for main_label in range(6):
#     for next_label in range(6):
#         for layer_num in range(5):
#             lama_cosine = []
#             if main_label != next_label:
#                 for l in range(len(reps_dict[main_label]['lama'][layer_num])):
#                     for k in range(len(reps_dict[next_label]['lama'][layer_num])):
#                         lama_cosine.append(get_cosine_similarity(reps_dict[main_label]['lama'][layer_num][l], reps_dict[next_label]['lama'][layer_num][k]))
            
#                 all_dict[main_label][next_label]['lama'][layer_num] = torch.tensor(lama_cosine)

In [36]:
# # i, j 레이블 간의 평균 코사인 유사도 구하기
# for i in range(6):
#     for j in range(6):
#         if i != j:
#             print(f'Label {i} - Label {j} _ Layer : {0}')
#             print(f'Lama mean : {all_dict[i][j]["lama"][0].mean()}, std : {all_dict[i][j]["lama"][0].std()}, min : {all_dict[i][j]["lama"][0].min()}, max : {all_dict[i][j]["lama"][0].max()}')
            
#             print(f'Label {i} - Label {j} _ Layer : {1}')
#             print(f'Lama mean : {all_dict[i][j]["lama"][1].mean()}, std : {all_dict[i][j]["lama"][1].std()}, min : {all_dict[i][j]["lama"][1].min()}, max : {all_dict[i][j]["lama"][1].max()}')

#             print(f'Label {i} - Label {j} _ Layer : {2}')
#             print(f'Lama mean : {all_dict[i][j]["lama"][2].mean()}, std : {all_dict[i][j]["lama"][2].std()}, min : {all_dict[i][j]["lama"][2].min()}, max : {all_dict[i][j]["lama"][2].max()}')

#             print(f'Label {i} - Label {j} _ Layer : {3}')
#             print(f'Lama mean : {all_dict[i][j]["lama"][3].mean()}, std : {all_dict[i][j]["lama"][3].std()}, min : {all_dict[i][j]["lama"][3].min()}, max : {all_dict[i][j]["lama"][3].max()}')

#             print(f'Label {i} - Label {j} _ Layer : {4}')
#             print(f'Lama mean : {all_dict[i][j]["lama"][4].mean()}, std : {all_dict[i][j]["lama"][4].std()}, min : {all_dict[i][j]["lama"][4].min()}, max : {all_dict[i][j]["lama"][4].max()}')


## Test  Data

In [37]:
reps_dict = {i : {'lama' : [],
                  'bert' : [],
                'roberta' : [] } for i in range(6)}
cnt = 0
for data in test_data:
    cnt += 1
    l_rep, b_rep, r_rep, label = data
    l_rep = l_rep.squeeze()
    b_rep = b_rep.squeeze()
    r_rep = r_rep.squeeze()
    label = label.item()
    reps_dict[label]['lama'].append(l_rep)
    reps_dict[label]['bert'].append(b_rep)
    reps_dict[label]['roberta'].append(r_rep)
cnt == len(test_data)

True

In [38]:
all_dict = {i : {j :{'lama' : [] ,'bert': [],'roberta': []}  for j in range(6) if i != j } for i in range(6)}
for i in range(6):
    # lama, bert, roberta 각각의 평균 코사인 유사도 구하기

    for j in range(6):
        lama_cosine = []
        bert_cosine = []
        roberta_cosine = []
        if i != j :
            for l in range(len(reps_dict[i]['lama'])):
                for m in range(len(reps_dict[j]['lama'])):
                    lama_cosine.append(get_cosine_similarity(reps_dict[i]['lama'][l].flatten() , reps_dict[j]['lama'][m].flatten()).item())
                    bert_cosine.append(get_cosine_similarity(reps_dict[i]['bert'][l].flatten() , reps_dict[j]['bert'][m].flatten()).item())
                    roberta_cosine.append(get_cosine_similarity(reps_dict[i]['roberta'][l].flatten() , reps_dict[j]['roberta'][m].flatten()).item())
            
            all_dict[i][j]['lama'] = torch.tensor(lama_cosine)
            all_dict[i][j]['bert'] = torch.tensor(bert_cosine)
            all_dict[i][j]['roberta'] = torch.tensor(roberta_cosine)
        else:
            continue

In [39]:
# i, j 레이블 간의 평균 코사인 유사도 구하기
for i in range(6):
    for j in range(6):
        if i != j:
            print(f'Label {i} - Label {j} :')
            print(f'Lama mean : {all_dict[i][j]["lama"].mean()}, std : {all_dict[i][j]["lama"].std()}, min : {all_dict[i][j]["lama"].min()}, max : {all_dict[i][j]["lama"].max()}')
            print(f'Bert mean : {all_dict[i][j]["bert"].mean()}, std : {all_dict[i][j]["bert"].std()}, min : {all_dict[i][j]["bert"].min()}, max : {all_dict[i][j]["bert"].max()}')
            print(f'Roberta mean : {all_dict[i][j]["roberta"].mean()}, std : {all_dict[i][j]["roberta"].std()}, min : {all_dict[i][j]["roberta"].min()}, max : {all_dict[i][j]["roberta"].max()}')
            print()

Label 0 - Label 1 :
Lama mean : 0.045304007828235626, std : 0.08481265604496002, min : -0.02327115461230278, max : 0.9857052564620972
Bert mean : -0.022854795679450035, std : 0.1350095123052597, min : -0.31750237941741943, max : 0.9513490796089172
Roberta mean : 0.15963636338710785, std : 0.3471395969390869, min : -0.23151761293411255, max : 0.9594566822052002

Label 0 - Label 2 :
Lama mean : -0.0009570687543600798, std : 0.01725848950445652, min : -0.05253167450428009, max : 0.03516114503145218
Bert mean : -0.31181126832962036, std : 0.11207077652215958, min : -0.4233130216598511, max : 0.20009838044643402
Roberta mean : -0.09491055458784103, std : 0.13751399517059326, min : -0.4239324927330017, max : 0.9356622695922852

Label 0 - Label 3 :
Lama mean : 0.017102360725402832, std : 0.02292558178305626, min : -0.020675532519817352, max : 0.14399981498718262
Bert mean : 0.03316356986761093, std : 0.11331013590097427, min : -0.25164732336997986, max : 0.3738418221473694
Roberta mean : -0.0

# llama layer별로 보기

In [40]:
reps_dict = {i : {'lama' : {j : [] for j in range(6)},
                  'bert' : [],
                'roberta' : [] } for i in range(6)}
cnt = 0
for data in train_data:
    cnt += 1
    l_rep, b_rep, r_rep, label = data
    l_rep = l_rep.squeeze()
    b_rep = b_rep.squeeze()
    r_rep = r_rep.squeeze()
    label = label.item()
    reps_dict[label]['lama'].append(l_rep)
    reps_dict[label]['bert'].append(b_rep)
    reps_dict[label]['roberta'].append(r_rep)
cnt == len(train_data)

AttributeError: 'dict' object has no attribute 'append'